## Section Headings Curation of El Salvador Policies

In this notebook there are a series of dictionaries and methods to curate section headings of El Salvador policies. Policies from El Salvador have a rather definite structure, so that the law text is organized under section headings. There are two kinds of sections, the ones that are general and that can be often found in many policies, and the ones which are specific. The sections headings which are more general often come with a whole range of name variants which makes the task of machine recognition difficult.

The goal of this notebook is to group all pretreatment methods that would harmonize sections heading to make the further processing machine friendly.

In [1]:
from pathlib import Path
import re, boto3, json, string, operator
import numpy as np

### Dictionaries of particular vocabularies to help in the curation of section headings

In [2]:
# Most policies come with the final signatures. This is a piece of text that we want to be able to recognize. To make the
# detection of signatures easier, this dictionary contain the most common terms that can be found in these lines of text.
official_positions = {"ALCALDE" : 0,
"Alcalde" : 0,
"MINISTRA" : 0,
"Ministra" : 0,
"MINISTRO" : 0,
"Ministro" : 0,
"PRESIDENTA" : 0,
"Presidenta" : 0,
"PRESIDENTE" : 0,
"Presidente" : 0,
"REGIDOR" : 0,
"Regidor"  : 0,
"REGIDORA" : 0,
"regidora" : 0,
"SECRETARIA" : 0,
"Secretaria" : 0,
"SECRETARIO" : 0,
"Secretario" : 0,
"SINDICA" : 0,
"Sindica" : 0,
"SINDICO" : 0,
"Sindico" : 0,
"VICEPRESIDENTA" : 0,
"Vicepresidenta" : 0,
"VICEPRESIDENTE" : 0,
"Vicepresidente" : 0
}
# This dictionary contains some correspondences among different text headings. This is under development and needs further
# improvement.The idea is to merge in a single name all the headings that point to the same conceptual concept. For example,
# "Definiciones" is a heading that can come alone or together with other terms so it can appear as "Definiciones básicas" or
# "Definiciones generales". With the dictionary we can fetch all headings that contain the word "Definiciones" and change the
# heading to "Definiciones".
merges = {
    "CONCEPTOS" : "DEFINICIONES",
    "DEFINICIONES" : "DEFINICIONES",
    "DISPOSICIONES FINALES" : "DISPOSICIONES GENERALES",
    "DISPOSICIONES GENERALES" : "DISPOSICIONES GENERALES",
    "DISPOSICIONES PRELIMINARES" : "DISPOSICIONES PRELIMINARES",
    "DISPOSICIONES REGULADORAS" : "DISPOSICIONES ESPECIALES",
    "DISPOSICIONES RELATIVAS" : "DISPOSICIONES ESPECIALES",
    "DISPOSICIONES ESPECIALES" : "DISPOSICIONES ESPECIALES",
    "DISPOSICIONES TRANSITORIAS" : "DISPOSICIONES GENERALES",
    "INFRACCIONES" : "INFRACCIONES",
    "INFRACCION ES" : "INFRACCIONES",
    "OBJETIVO" : "OBJETO",
    "OBJETO" : "OBJETO",
    "OBLIGACIONES" : "OBLIGACIONES Y PROHIBICIONES",
    "OBLIGACIONE" : "OBLIGACIONES Y PROHIBICIONES",
    "OBLIGACION" : "OBLIGACIONES Y PROHIBICIONES",
    "OBLIGATORIEDAD" : "OBLIGACIONES Y PROHIBICIONES",
    "POR TANTO" : "POR TANTO",
    "POR LO TANTO" : "POR TANTO",
    "PROHIBICIONES" : "OBLIGACIONES Y PROHIBICIONES",
    "PROHIBICION" : "OBLIGACIONES Y PROHIBICIONES"
}
# Eventhough the general gramar rule in Spanish is not to accent uppercase, there are many cases where a word in a heding might
# appear accented. This is a dictionary to armonize all headings without accents. The list is rather comprehensive, but there is
# still room for improvement.
# If we find some bug beyond simple misspelling which will be solved by spell checker, we can include it here. The example is in
# the first row with "ACTIVIDADESUSOS" which was found several times in headings.
bugs = {"ACTIVIDADESUSOS" : "ACTIVIDADES DE USOS"}

### Connection to the AWS S3 bucket
To effectively run this cell you need Omdena's credentials. Please keep them local and do not sync them in GitHub repos nor cloud drives. Before doing anything with this json file, please think of security!!

In [3]:
# json_folder = Path("C:/Users/user/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Notebooks/credentials/")
json_folder = Path("C:/Users/jordi/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Notebooks/credentials/")
filename = "Omdena_key.json"
file = json_folder / filename

with open(file, 'r') as f:
    key_dict = json.load(f) 

for key in key_dict:
    KEY = key
    SECRET = key_dict[key]

s3 = boto3.resource(
    service_name = 's3',
    region_name = 'us-east-2',
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)

### Functions and regular expressions

In [190]:

# Function to calculate the uppercase ratio in a string. It is used to detect section headings
def uppercase_ratio(string):
    return(len(re.findall(r'[A-Z]',string))/len(string))

# Regular expression to clear html tags (here is basically to remove the page tags)
cleanr = re.compile(r'<.*?>')
# Te function to clear html tags
def clean_html_tags(string):
  return cleanr.sub('', string)

# Function to remove the last lines of a document, the ones that contain the signatures of the officials. It depends on the
# dictionary "official_positions"
def remove_signatures(line):
    signature = False
    for key in official_positions:
        if key in line:
            signature = True
            break
    return signature

# Function to change accented words by non-accented counterparts. It depends on the dictionary "accent_marks_bugs" 
accents_out = re.compile(r'[áéíóúÁÉÍÓÚ]')
accents_dict = {"á":"a","é":"e","í":"i","ó":"o","ú":"u","Á":"A","É":"E","Í":"I","Ó":"O","Ú":"U"}
def remove_accents(string):
    for accent in accents_out.findall(string):
        string = string.replace(accent, accents_dict[accent])
    return string

# Function to merge headlines expressing the same concept in different words. It depends on the dictionary "merges"
def merge_concepts(line):
    for key in merges:
        if key in line:
            line = merges[key]
            break
    return line

def clean_bugs(line):
    for key in bugs:
        if key in line:
            line = line.replace(key, bugs[key])
    return line

# Function to add items to the dictionary with duplicate removal
def add_to_dict(string, dictionary, dupl_dict):
    if string in dupl_dict or string == None:
        pass
    else:
        dupl_dict[string] = 0
        if string in dictionary:
            dictionary[string] = dictionary[string] + 1
        else:
            dictionary[string] = 1
    return dictionary

clean_acron = re.compile(r'(A\s*\.M\s*\.)|(\bArt\s*\.)|(\bART\s*\.)|(\bArts\s*\.)|(\bAV\s*\.)|(\bDr\s*\.)|(\bIng\s*\.)|(\bLic\s*\.)|(\bLicda\s*\.)|(\bLIC\s*\.)|(mts\s*\.)|(\bNo\s*\.)|(P\s*\.M\s*\.)|(prof\s*\.)|(profa\s*\.)|(sp\s*\.)|(ssp\s*\.)|(to\s*\.)|(ta\s*\.)|(var\s*\.)')  
def clean_acronyms(line):
    acro = clean_acron.findall(line)
    for item in acro:
        for acronym in item:
            if acronym != '':
                line = line.replace(acronym, clean_punct.sub('', acronym))
    return line
whitespaces = re.compile(r'[ ]{2,}')
def clean_whitespace(line):
    return whitespaces.sub(' ', line)

decimal_points = re.compile(r'(\b\d+\s*\.\s*\d+)')
def change_decimal_points(line):
    dec = decimal_points.findall(line)
    for decimal in dec:
        if decimal != '':
#             print(decimal)
            line = line.replace(decimal, clean_punct.sub(',', decimal))
    return line
                
# Regular expression to clear punctuation from a string
clean_punct = re.compile('[%s]' % re.escape(string.punctuation))
# Regular expression to clear words that introduce unnecessary variability to headings. Some regex still not work 100% we need
# to improve them.
clean_capitulo = re.compile(r'(APARTADO \S*)|(APARTADO\s)|(^ART\.\s*\S*)|(^Art\.\s*\S*)|(^Arts\.\s*\S*)|(Capítulo \S*)|(CAPITULO \S*)|(CAPITULO\S*)|(CAPÍTULO \S*)|(CAPITULÓ \S*)|(CAPITULOS \S*)|(CAPITUO \S*)|(CATEGORIA\b)|(CATEGORÍA\b)|(SUBCATEGORIA\b)|(SUBCATEGORÍA\b)|(TITULO\s\S*)|(TÍTULO\s\S*)')
clean_bullet_point = re.compile(r'\b[A-Za-z]\s*\. |\b[A-Za-z]\s*\.\s*[B-Za-z]\b|\b[A-Z]+\s*\.|^\d+\s*\.\s*\D+')

# Function sentence
def clean_sentence(string):
    string = clean_capitulo.sub('', string)
    string = clean_bullet_point.sub('', string).rstrip().lstrip()
#     string = clean_punct.sub('', string).rstrip().lstrip()
    if string != "":
        return string
    else:
        return None    
    
# points = re.compile(r'(\b\w+\s*\.\s*\b[^\d\W]+)')
# def check_points(line):
#     return points.findall(line)
#     print(points.findall(line))

points = re.compile(r'(\b\w+\b\s*){3,}')
def check_sentence(line):
    if points.findall(line):
        return True
    else:
        return False

def split_into_sentences(line):
    sentence_list = []
    for sentence in line.split("."):
        if check_sentence(sentence):
            sentence = sentence.rstrip().lstrip()
            sentence_list.append(sentence)
    return sentence_list

In [193]:
test_string = "A. Hola, em dic Jordi. B. No sé massa perquè l'Art . 22 conté 22.34€. Tanmateix sembla que la Licda. una cosa. voldria  55.22. no fotis"
# test_string = "Prova senzilleta per veure què passa si no hi ha punt"
test_string = clean_sentence(test_string)
test_string = clean_acronyms(test_string)
test_string = clean_whitespace(test_string)
test_string = change_decimal_points(test_string)
print(test_string)
sentences = []
[sentences.append(sentence) for sentence in split_into_sentences(test_string)]
print(sentences)
# print(sentences)

# if check_sentence(test_string):
#     

Hola, em dic Jordi. No sé massa perquè l'Art 22 conté 22,34€. Tanmateix sembla que la Licda una cosa. voldria 55,22. no fotis
['Hola, em dic Jordi', "No sé massa perquè l'Art 22 conté 22,34€", 'Tanmateix sembla que la Licda una cosa']


### Pipeline to process files from S3 bucket
By executing this cell you will go through all policies in El Salvador and process section headings that will be saved in a dictionary. This should be merged with the notebook that builds up the final json files out of plain txt files.

In [134]:
folder = "text-extraction/"
filename = "00a55afe4f55256567397a68df5d7f97e642480b" # This is only if you want to test on a single file
bag_of_words = {}
sentences = []
sentences_dict = {}

i = 0
for obj in s3.Bucket('wri-latin-talent').objects.all().filter(Prefix='text-extraction'):
    if folder in obj.key and obj.key.replace(folder, "") != "": # and filename in obj.key # Un comment the previous string to run the code just in one sample document.
#         print(i, "**", obj.key)
        file = obj.get()['Body'].read().decode('utf-8')  #get the file from S3 and read the body content
        lines = file.split("\n") # Split by end of line and pipe lines into a list
        duplicates_dict = {} #Sometimes the same heading can be found more than once in a document. This will help on removing them
        for line in lines:
            if uppercase_ratio(clean_html_tags(line)) > 0.6 and len(line) > 6:
                if remove_signatures(line):
                    break
                else:
                    line = clean_html_tags(line)
                    line = remove_accents(line)
                    line = clean_bugs(line)
                    line = clean_sentence(line)
                    if line == None:
                        continue
                    line = merge_concepts(line)
                    bag_of_words = add_to_dict(line, bag_of_words, duplicates_dict)
            else:
                line = clean_html_tags(line)
                line = remove_accents(line)
                line = clean_bugs(line)
                line = clean_acronyms(line)
                line = clean_whitespace(line)
                line = clean_sentence(line)
                if line == None:
                    continue
                for sentence in split_into_sentences(line):
                     
#                 sentences.append(line)
#                 line_breaks = check_points(line)
#                 if line_breaks:
# #                     print(line_breaks)
#                     for item in line_breaks:
#                         sentences_dict[item] = 0
#                 print("--", line)
#             s3.Object('wri-latin-talent', key).put(Body = content)#This will save all the contents in the string variable "content" into a txt file in the Pre-processed folder
        i += 1

In [135]:
print(len(sentences_dict))
for k in sorted(sentences_dict):
    print(k, ":", sentences_dict[k])

4912
0 . Ll : 0
0 . a : 0
0 . o : 0
0. Se : 0
0. ua : 0
00 . En : 0
00 . No : 0
00 . a : 0
00 . horas : 0
00 . hrs : 0
00. Dolares : 0
00. En : 0
00. Las : 0
00. Para : 0
00. Por : 0
00. Tasa : 0
00. hrs : 0
01 . a : 0
01 . horas : 0
01. Por : 0
04. Por : 0
04. mts : 0
05. Por : 0
06 . y : 0
07. mts : 0
08. Por : 0
1 . Y : 0
1 . i : 0
1 . o : 0
1.  Botar : 0
1.  Revegetacion : 0
1. Alos : 0
1. Amonestacion : 0
1. Aplicar : 0
1. Arboles : 0
1. Area : 0
1. Asimismo : 0
1. Año : 0
1. Baterias : 0
1. Calificacion : 0
1. Colocar : 0
1. Con : 0
1. Concientizar : 0
1. Conservar : 0
1. Controlar : 0
1. Corredor : 0
1. Crease : 0
1. Cuando : 0
1. De : 0
1. Debe : 0
1. Declarase : 0
1. Delimitacion : 0
1. Denominados : 0
1. Desarrollo : 0
1. Director : 0
1. Disposiciones : 0
1. ESPECIES : 0
1. Edificaciones : 0
1. El : 0
1. Elaborar : 0
1. Elfuncionamiento : 0
1. En : 0
1. Es : 0
1. Escritura : 0
1. Especies : 0
1. Esta : 0
1. Establecer : 0
1. Establecese : 0
1. Evitar : 0
1. Factibilidad : 0
1

37. Se : 0
37.Las : 0
38. Competencias : 0
38. Cuando : 0
38. El : 0
38. Las : 0
38. Legislacion : 0
38. Los : 0
38. Se : 0
38. Supletoriedad : 0
386. Sin : 0
39. A : 0
39. Al : 0
39. Derogaciones : 0
39. El : 0
39. Las : 0
39. Procedimientos : 0
39. Se : 0
396. Santa : 0
3er. Regidor : 0
4.  Alterar : 0
4.  Elseñor : 0
4. A : 0
4. Asimismo : 0
4. Azucar : 0
4. Clausura : 0
4. Competencia : 0
4. Coordinadores : 0
4. Cualquier : 0
4. DE : 0
4. Dar : 0
4. Debe : 0
4. Definiciones : 0
4. Del : 0
4. Descargar : 0
4. Durante : 0
4. Edificaciones : 0
4. El : 0
4. En : 0
4. Es : 0
4. Gozaran : 0
4. Identificar : 0
4. Incorporase : 0
4. Incremento : 0
4. Industria : 0
4. Informarse : 0
4. Infraestructuras : 0
4. Inspecciones : 0
4. Instalacion : 0
4. Instalaciones : 0
4. Instrumentos : 0
4. Justificacion : 0
4. La : 0
4. Lagos : 0
4. Las : 0
4. Laspracticas : 0
4. Limpiar : 0
4. Los : 0
4. Mantener : 0
4. No : 0
4. Obras : 0
4. PLAN : 0
4. Para : 0
4. Participar : 0
4. Pegar : 0
4. Permitir : 

Il. De : 0
Il. El : 0
Il. Fuentes : 0
Il. Infracciones : 0
Il. La : 0
Il. Lugar : 0
Il. Nombre : 0
Il. PORCION : 0
Il. Promover : 0
Il. Que : 0
Il. Ruido : 0
Il. Sesiones : 0
Il. Ubicacion : 0
Il. Zonas : 0
Il. Zonificacion : 0
Ill. Caracterizacion : 0
Ill. Identificacion : 0
Ill. La : 0
Ill. Planes : 0
Ill. Presentacion : 0
Ill. Que : 0
Ill. Realizar : 0
Impermeabilizada. Esta : 0
Implementacion. En : 0
Imposible. y : 0
Incineracion. b : 0
Industria. i : 0
Industrial. ACTIVIDADES : 0
Industrial. Definiciones : 0
Industriales. Los : 0
Industriales. f : 0
Industrias. Hospitales : 0
Informacion. Planos : 0
Informacion. c : 0
Infracciones. Art : 0
Infracciones. Las : 0
Infraestructura. ACTIVIDADES : 0
Infraestructura. Se : 0
Infraestructura. Son : 0
Institucional. ACTIVIDADES : 0
Intensiva. Forman : 0
Internacionales. ACTIVIDADES : 0
Internacionales. Dichas : 0
Interno. Tales : 0
Isidro. Unidad : 0
Jaltepeque. AL : 0
Jaltepeque. Dejando : 0
Jocotal. Asi : 0
Jordan. El : 0
Juayua. a : 0
Ju

año. f : 0
años. Desde : 0
años. Dicha : 0
años. El : 0
años. En : 0
baja. Se : 0
baja. b : 0
baldios. Es : 0
baldios. Se : 0
barbilla. f : 0
barril. Igualmente : 0
barril. Y : 0
basicas. Art : 0
basicas. Para : 0
basico. Cuando : 0
basicos. Art : 0
basicos. Estos : 0
basicos. Se : 0
basura. Asi : 0
basura. Descubrir : 0
basura. Tambien : 0
basura. c : 0
basuras. No : 0
be .uy : 0
biblioteca. A : 0
biblioteca. Dicha : 0
biblioteca. Los : 0
bibliotecologia. Para : 0
bicicletas. Durante : 0
bicicletas. Se : 0
bienes. Ademas : 0
bienes. Permiten : 0
bienes. b : 0
biodegradables. El : 0
biodiversidad. Desarrollar : 0
biodiversidad. Impulsandose : 0
biologica. Tambien : 0
biologicas. En : 0
biologicos. Ademas : 0
bisturi. Asi : 0
bloque. Detallan : 0
bombero. Reclusorios : 0
bosque. Terrenos : 0
bosque. b : 0
bosques. Ademas : 0
breakfast. ACTIVIDADES : 0
breakfast. Actividades : 0
cadena. La : 0
caducara. Por : 0
cafes. ACTIVIDADES : 0
cafeteria. ACTIVIDADES : 0
calendarios. Si : 0
calibra

general. Tendran : 0
general. b : 0
genetica. f : 0
geodesicas. li : 0
geodesicos. Esquema : 0
geologicas. La : 0
gestion. Debe : 0
gestion. d : 0
global. Entre : 0
global. Entres : 0
gob.sv : 0
grabacion. Esto : 0
graves. Art : 0
graves. En : 0
graves. Graves : 0
graves. Infracciones : 0
graves. Las : 0
graves. Para : 0
graves. Son : 0
graves. a : 0
graves. b : 0
guantes. g : 0
guapa. Y : 0
guarderias. Centros : 0
guiones. De : 0
habiles. Art : 0
habiles. En : 0
habiles. Para : 0
habiles. Si : 0
habiles. Sin : 0
habiles. Transcurrido : 0
habitacional. Manuel : 0
habitacional. Se : 0
habitacionales. Incluye : 0
habitacionales. Quedan : 0
habitacionales. y : 0
habitado. Los : 0
habitante. Art : 0
habitante. En : 0
habitantes. Con : 0
habitantes. De : 0
habitantes. El : 0
habitantes. Estas : 0
habitantes. Las : 0
habitantes. con : 0
habitantes. de : 0
hacer. Sin : 0
hacienda. Asi : 0
hacienda. LINDERO : 0
hecho. De : 0
hectarea. Dentro : 0
hectarea. b : 0
hectarea. d : 0
hectareas. El : 

plan. Tambien : 0
plantacion. el : 0
plantas. Edificaciones : 0
planteado. El : 0
planteado. Transcurrido : 0
plasticas. Esta : 0
plastico. Generalmente : 0
playas. Quebradas : 0
playas. Seran : 0
playas. Vencido : 0
plaza. ACTIVIDADES : 0
plazas. Durante : 0
plazas. Para : 0
plazas. parques : 0
poblacion. C : 0
poblacion. Deberan : 0
poblacion. Dicha : 0
poblacion. En : 0
poblacion. Estos : 0
poblacion. La : 0
poblacion. Para : 0
poblacion. Por : 0
poblacion. Toda : 0
poblacion. Y : 0
poblacionales. Para : 0
pobladores. Ademas : 0
poda. Por : 0
podar. e : 0
polideportivos. Actividades : 0
poligono. b : 0
poligonos. Las : 0
poniente. LINDERO : 0
por. escrito : 0
por.las : 0
porcentaje. Ademas : 0
poseedor. es : 0
posesion. No : 0
posteriormente. El : 0
posteriormente. Se : 0
potable. Esta : 0
potable. N : 0
practicas. Debiendo : 0
preceptivo. Las : 0
precio. Este : 0
preferencial. Dicho : 0
preferente. Este : 0
presas. Tecnica : 0
presentacion. Asi : 0
presentan. Tambien : 0
presentar.

vigente. El : 0
vigente. En : 0
vigente. Permiso : 0
vigentes. Dicha : 0
vigentes. Para : 0
vigentes. Por : 0
vii. Extension : 0
viii. Especificacion : 0
visitado. Tiene : 0
visuales. Ademas : 0
visuales. Se : 0
vivas. b : 0
vivienda. Comercializacion : 0
viviendas.  Plano : 0
viviendas. Deberan : 0
viviendas. En : 0
vo.pua : 0
www.marn : 0
zafra. Las : 0
zancudo. Mantener : 0
zancudo. TRANSFORMADAS : 0
zancudos. AEDES : 0
zancudos. Abate : 0
zancudos. Aquellos : 0
zancudos. Botar : 0
zancudos. Los : 0
zancudos. Si : 0
zona. Art : 0
zona. Condiciones : 0
zona. Considerando : 0
zona. Constan : 0
zona. En : 0
zona. Estos : 0
zona. Fosa : 0
zona. La : 0
zona. Las : 0
zona. Una : 0
zonas. En : 0
zonas. Evitando : 0
zonas. La : 0
zonas. Podra : 0
zonas. Podran : 0
zonas. Todos : 0
zoosanitarios. Su : 0


#### Short summary

In [ ]:
print("After preprocessing there are {} different headings in El Salvador policies".format(len(bag_of_words)))
print("{} documents have been processed".format(i))
print("There are {} lines of text as sentences".format(len(sentences)))

#### Dictionary items sorted by occurrence

In [ ]:
dict( sorted(bag_of_words.items(), key=operator.itemgetter(1),reverse=True))

#### Dictionary items sorted by heading text

In [ ]:
for k in sorted(bag_of_words):
    print(k, ":", bag_of_words[k])

#### Saving sentences as csv

In [ ]:
print(sentences[0:2])

In [ ]:
# path = Path("C:/Users/user/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Notebooks/Data/")
path = Path("C:/Users/jordi/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Notebooks/Data/")
filename = "sentences.npy"
file = path / filename
np_sentences = np.array(sentences)
with open(file, 'wb') as f:
    np.save(f, np_sentences)

#### Pipeline to process one file from HD folder
This is a pipeline to process a test file in a local folder.

In [ ]:
data_folder = Path("../Documents_de_mostra/")
filename = "00a55afe4f55256567397a68df5d7f97e642480b.pdf.txt"
bag_of_words = {}

i = 0
file = data_folder / filename
with open(file, 'r', encoding = 'utf-8') as file:
    lines = file.readlines()
    duplicates_dict = {}
    for line in lines:
        line = clean_html_tags(line)
        if uppercase_ratio(line) > 0.6 and len(line) > 6:
            if remove_signatures(line):
                break
            else:
#                 print(line)
                line = remove_accents(line)
                clean_line = clean_headings(line)
                bag_of_words = add_to_dict(clean_line, bag_of_words, duplicates_dict)
#                 print(clean_line)
        i += 1
#     data = file.read().replace('\n', '')

In [ ]:
bag_of_words